<h1>Kapitel 11: Funktionen (Teil 3): Verschachtelung und funktionale Programmierung</h1>
<p>Grundlagen der Programmierung 1<p><h2>Holger Karl</h2>
 

<h1>Overview</h1> 

1. Überblick
1. Verschachtelte Funktionen
1. Closures
1. Anonyme Funktionen: lambda-Ausdrücke
1. Funktionaler Programmierstil
1. Generatoren
1. Decorators
1. Zusammenfassung 
 


<h1>Overview</h1> 

1. **<font color="red">Überblick</font>**
1. Verschachtelte Funktionen
1. Closures
1. Anonyme Funktionen: lambda-Ausdrücke
1. Funktionaler Programmierstil
1. Generatoren
1. Decorators
1. Zusammenfassung

# Setup

Main point here is to set up warnings properly for presentation and to
load tutormagic, so that we can later on use pythontutor for code
animations. 

In [ ]:
%load_ext tutormagic
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning, module='.*/IPython/.*')

# Was bisher geschah

-   Wir haben uns grundlegende Techniken zur Gestaltung und Nutzung von
    Funktionen angeschaut
-   Wir haben Funktionen im Kontext von Klassen als Methoden verwendet
    (eine Funktion, die an/mit einem Objekt arbeitet)
-   Wir haben angedeutet, dass eine Funktion eigentlich auch nur ein
    Objekt (von einem recht speziellen Typ) ist

# Dieses Kapitel

-   Wir erweitern die Möglichkeiten, wie man mit Funktionen arbeiten
    kann
    -   Insbes. die Vorstellung, dass eine Funktion ein Objekt ist, auf
        das man eine Referenz erzeugen und herumreichen kann
-   Darauf aufbauend erarbeiten wir typische Programmiertechniken zur
    *funktionalen Programmierung*

<h1>Overview</h1> 

1. Überblick
1. **<font color="red">Verschachtelte Funktionen</font>**
1. Closures
1. Anonyme Funktionen: lambda-Ausdrücke
1. Funktionaler Programmierstil
1. Generatoren
1. Decorators
1. Zusammenfassung

# Vorüberlegung: `def` ist eine Anweisung

-   `def` ist eine Anweisung, die eine Funktion mit einem Namen versieht
    -   Der Code der Funktion wird intern abgelegt
    -   In einem Objekt einer geeigneten Klasse gespeichert
-   Im Block einer Funktion stehen Anweisungen
-   Darf dann dort auch ein `def` stehen?

# `def` in Funktionsblock

Syntaktisch geht das natürlich &#x2013; Semantik? 

In [ ]:
%%tutor -t 
def f():
    print("f1") 

    def g():
        print("g!")

    g()
    print("f2") 

f()
print("nach f")

# `def` in Funktionsblock: Semantik

Bei geschachtelter Funktionsdefinition: 

-   Der Name der inneren Funktion ist im Namensraum der äußeren Funktion
    nach Ausführung der inneren `def`-Anweisung bekannt
-   Dieser Name kann &#x2013; wie jeder andere Funktionsname auch &#x2013; zum Aufruf
    benutzt werden
-   Nach Rückkehr der äußeren Funktion verschwindet der Name `g`
    -   Im Beispiel: damit ist diese Funktion auch nicht aufrufbar!

# Verschachtelte Funktionen: Argumente, Rückgabewert

Argumentübergabe und Rückgabewert funktionieren ganz normal 

In [ ]:
def f():
    def g(x):
        print(x+1)
        return x+2

    y = g(22)
    print(y)

f()

# Verschachtelte Funktionen: Zugriff auf Name?

-   Wie funktioniert der Zugriff auf Namen außerhalb des eigenen
    Namensraum aus einer inneren Funktion heraus?
-   Im wesentlichen: Regeln wie bisher

# Zugriff auf globale Namen, lesend

Ohne besondere Vorkehrungen, wenn nur lesen 

In [ ]:
def f():
    def g():
        print(x)
    g()

x = 1
f()

# Zugriff auf globale Namen, schreiben

Schreibender Zugriff: Erfordert Deklaration mit `global`

In [ ]:
def f():
    def g():
        global x
        x = x + 1 
        print(x)
    g()

x = 1
f()

# Zugriff auf Namen der äußeren Funktion? `global`?

Spannender Punkt: Was ist mit Namen, die nicht global, aber in der
äußeren Funktion vereinbart ist? 

## Zugriff mit `global`?<a id="orgheadline19"></a>

In [ ]:
def f():
    def g():
        global x
        x = x + 1 
        print(x)
    x = 1
    g()

f()

# Zugriff auf Namen der äußeren Funktion? Direkt?

In [ ]:
def f():
    def g():
        x = x + 1 
        print(x)
    x = 1
    g()

f()

# Zugriff auf Namen der äußeren Funktion &#x2013; Reihenfolge?

Liegt es vielleicht an der Reihenfolge? 

-   Im Beispiel eben war bei der *Definition* von `g` der Namen `x`
      noch gar nicht existent
-   Muss es das `x` schon geben, wenn `g` definiert wird? Damit `g`
      darauf zugreifen kann?

In [ ]:
def f():
    x = 1 
    def g():
        print(x)
    g()

f()

## Ja!<a id="orgheadline22"></a>

Offenbar wird auch der Namensraum einer umschließenden Funktion
durchsucht! 

# Modifizierender Zugriff auf Namen der äußeren Funktion?

Kann man solche "umschließenden Namen" auch verändern? 

-   Kurz für "Namen aus umschließendem Namensraum"

In [ ]:
def f():
    x = 1 
    def g():
        x += 1
        print(x)
    g()

f()

## Offenbar nein<a id="orgheadline24"></a>

Das ist analog zur Situation bei `global`

# Neues Schlüsselwort: `nonlocal`

Analog zu `global`: Neues Schlüsselwort, um umschließende Namen zu
verändern

-   `nonlocal`
-   Unterschied zu `global`: Der Name muss schon existieren! (durch
    Zuweisung, siehe oben)

In [ ]:
def f():
    x = 1 
    def g():
        nonlocal x
        x += 1
        print(x)
    g()

f()

# Beispiel: Tiefere Verschachtelung (1)

In [ ]:
def f():
    x = 1 
    def g():
        def h():
            nonlocal x
            x += 1
            print("x in h:", x)
        h()
        print("x in g:", x)
    g()
    print("x in f:", x)

f()

# Beispiel: Tiefere Verschachtelung (2)

In [ ]:
%%tutor -t
def f():
    x = 1 
    def g():
        x = 42
        def h():
            nonlocal x
            x += 1
            print("x in h:", x)
        h()
        print("x in g:", x)
    g()
    print("x in f:", x)

f()

## Anmerkung: `x` aus `f`?<a id="orgheadline28"></a>

Es gibt keine Möglichkeit, von `h` aus auf das von `f` angelegte `x`
zuzugreifen. Dieses `x` wird durch das `x` von `g` verdeckt! 

# Nachschlageregel: LEGB

-   Bisher einfache Nachschlagregel für Namensräume: Erst lokal, dann global
-   Tatsächlich etwas umfangreicher

<div class="definition-head" style="border:3px; border-style:solid; border-color:#5da9e9; padding: 5px; background-color:#5da9e9;  line-height:1.5em; text-align:center; border-radius: 15px 15px 0px 0px; margin-bottom: 0; width: 100%;"><b>Definition: Nachschlageregel für Namen: LEGB<a id="orgheadline30"></a></b></div>

Python sucht Namen in diesen Namensräumen, in dieser Reihenfolge: 

1.  dem lokalen Namensraum (einer Funktion): **L**
2.  dem Namensraum umgebender (*enclosing*) Funktionen: **E**
3.  dem globalen Namensraum: **G**
4.  dem Namensraum eingebauter (vordefinierter) Namen (build-in): **B**

Kurz: Die Reihenfolge ist **LEGB** 

<div class="definition-content" style="border:3px; border-style:solid; border-color:#5da9e9; padding: 5px; margin-top: 0; border-radius: 0px 0px 15px 15px; width: 100%; text-align:left"></div>

# Nachschlageregel: LEGB &#x2013; Illustration

<center>![img](./uml/legb.png "Nachschlageregel LEGB")</center>

<h1>Overview</h1> 

1. Überblick
1. Verschachtelte Funktionen
1. **<font color="red">Closures</font>**
1. Anonyme Funktionen: lambda-Ausdrücke
1. Funktionaler Programmierstil
1. Generatoren
1. Decorators
1. Zusammenfassung

# Vorüberlegung: Referenzen auf Funktionen?

-   Kann man Funktionsnamen wie normale Namen behandeln?
-   Kann man diese kopieren, und trotzdem noch aufrufen?

In [ ]:
def f():
    print("f!")

g = f
g()

## Was passiert hier?<a id="orgheadline33"></a>

-   `f` ist ein Name für ein Funktionsobjekt
-   Die Anweisung `g = f` lässt den Namen `g` auf das gleiche Objekt
    referenzieren
-   Also wird der Aufruf `g()` auch die gleiche Funktion aufrufen
    -   Der referenzierende Name  ist *irrelevant*!

# Vorüberlegung 2: Funktionsreferenzen als Rückgabewert?

-   Wenn man Funktionsreferenzen kopieren kann
-   Dann kann man sie doch bestimmt auch als Rückgabewert einer Funktion
    verwenden?

In [ ]:
%%tutor -t
def f():
    def g():
        print("innere Funktion!")
    return g

h = f()
h()

## Natürlich!<a id="orgheadline35"></a>

-   Die Semantik ist ganz banal!
-   Nach Aufruf von `f` ist `h` ein
    weiterer Name für das Funktionsobjekt geworden, das ursprünglich
    unter dem Namen `g` eingeführt wurde

# Analogie: Normales Objekt als Rückgabewert

Zur Analogie: Dieser Code ist ja klar 

-   Eine Liste kann man nicht aufrufen, sondern ausgeben
-   Aber sonst ist das *genau das gleiche*!

In [ ]:
def f():
    g = ["ein", "Beispiel", "Objekt"]
    return g

h = f()
print(h)

# Funktionen mit Parametern als Rückgabe?

Was passiert, wenn wir eine Funktion zurückgeben, die
Parameter benutzt? 

In [ ]:
def f():
    def g(x):
        print("innere Funktion!", x)
    return g

h = f()
h("Wert für x")

## Parameter bleibt bestehen<a id="orgheadline38"></a>

Dann hat die Funktion natürlich einen Parameter 

-   Aufruf unter neuem Namen erfordert diesen Parameter

# Funktion mit Zugriff auf nichtlokalen Namen als Rückgabe

Spannender: 

-   Innere Funktion greift auf Parameter der äußeren Funktion zu
-   Innere Funktion wird zurückgeben
-   und dann aufgerufen

Erwartetes Verhalten? 

In [ ]:
%%tutor -t
def f():
    n = 5
    def g(x):
        print("innere Funktion!", x*n)
    return g

h = f()
h("Wert für x")

## Äußerer Parameter bleibt erhalten!<a id="orgheadline40"></a>

-   Offenbar behält die Funktion, die unter `h` im globalen Scope
    bekannt ist, Zugriff auf den Namen `n`
-   **Erstaunlich**! `n` im Namensraum von `f`, aber `f` schon beendet!?

# Nicht-lokaler Name selbst als Parameter

-   In obigem Beispiel: `n` hatte einen festen Wert
-   Aber wo der Wert für `n` herkommt ist egal
    -   Er muss lediglich vor dem `def` von `g` vorhanden sein
-   Es könnte selbst ein Funktionsparameter sein?

In [ ]:
def f(n):
    def g(x):
        print("innere Funktion!", x*n)
    return g

h = f(3)
h("Wert für x")

# Beispiel &#x2013; Aufgeräumt

In [ ]:
%%tutor -t
def vervielfacher_fabrik(n):
    def vervielfacher(x):
        return x*n

    return vervielfacher

doppler = vervielfacher_fabrik(2)
fuenffacher = vervielfacher_fabrik(5) 

print(doppler("Mehrfach ausgeben! "))
print(fuenffacher("Mehrfach ausgeben! "))

# Muster: Fabrik (*Factory*)

-   Beispiel hatte mehrere Bestandteile
    -   Eine `...-fabrik` Funktion: Sie erzeugt eine andere Funktion,
        richtig parametriert
    -   Der Bauplan für die zu erzeugende Funktion (hier: `vervielfacher`)
        -   Bauplan wird konkretisiert durch Einsetzen von Werten für
            Parameter (hier: `n`)
    -   Mehrfacher Aufruf der Fabrik mit unterschiedlichen Parametern
        -   Ergibt maßgeschneiderte Funktionen, die aufgerufen werden
-   Das sog. *factory pattern*

# Realisierung: Closures

-   Das erstaunliche: Die produzierten Funktionen merken sich die Werte
    der Parameter der Fabrik-Funktion!
-   Dazu werden die Frames (Namensräume) dieser Fabrikfunktionsaufrufe
    nicht entsorgt, sondern an die produzierte Funktion (`doppler`,
    `fuenffacher`, &#x2026;) gekoppelt
-   Diese Datenstruktur (Kopplung von Frames an einen Funktionsnamen)
    heißt *Closure*
-   Closure merken sich den relevanten Zustand &#x2013; ein *state retention*
      Ansatz

<div class="definition-head" style="border:3px; border-style:solid; border-color:#5da9e9; padding: 5px; background-color:#5da9e9;  line-height:1.5em; text-align:center; border-radius: 15px 15px 0px 0px; margin-bottom: 0; width: 100%;"><b>Definition: Closure<a id="orgheadline45"></a></b></div>

Ein Closure ist eine Funktion, deren nicht-lokale Namen an Objekte
gebunden wurden. 

<div class="definition-content" style="border:3px; border-style:solid; border-color:#5da9e9; padding: 5px; margin-top: 0; border-radius: 0px 0px 15px 15px; width: 100%; text-align:left"></div>

# Nutzen: Fabriken und Closures

-   Beispiel oben ist zugeben etwas künstlich
-   Typischer Einsatz von Closures:
    -   Graphische Nutzerschnittstellen: Funktionen, die abhängig von
        Nutzereingabe erzeugt werden
    -   Web-Anwendungen: Funktionen erzeugen, die auf Veränderungen
        reagiert (sog. *callbacks*)
        -   Häufig in event handlers in Javascript!

<h1>Overview</h1> 

1. Überblick
1. Verschachtelte Funktionen
1. Closures
1. **<font color="red">Anonyme Funktionen: lambda-Ausdrücke</font>**
1. Funktionaler Programmierstil
1. Generatoren
1. Decorators
1. Zusammenfassung

# Funktionsdefinition bisher: `def`

-   `def` ist Anweisung
    -   Erzeugt ein Funktionsobjekt
    -   Legt einen Namen im entsprechenden Namensraum an
    -   Verbindet Namen mit erzeugtem Objekt
-   Was, wenn man den Namen eigentlich gar nicht braucht?
    -   Sondern nur das Funktionsobjekt an sich?
    -   Z.B., weil es in einer Factory sowieso direkt zurückgegeben wird?
    -   Oder nur an einer einzigen Stelle gebraucht wird, z.B. in einer
        list comprehension?

# Funktionsname in Factory: Nötig?

Beispiel oben: Name `vervielfacher` eigentlich nicht sehr nützlich  

In [ ]:
def vervielfacher_fabrik(n):
    def vervielfacher(x):
        return x*n

    return vervielfacher

doppler = vervielfacher_fabrik(2)
fuenffacher = vervielfacher_fabrik(5) 

print(doppler("Mehrfach ausgeben! "))
print(fuenffacher("Mehrfach ausgeben! "))

# Anonyme Funktionen: &lambda;-Ausdrücke

-   Das kommt häufig vor, verdient also Sprachunterstützung

<div class="definition-head" style="border:3px; border-style:solid; border-color:#5da9e9; padding: 5px; background-color:#5da9e9;  line-height:1.5em; text-align:center; border-radius: 15px 15px 0px 0px; margin-bottom: 0; width: 100%;"><b>Definition: &lambda;-Ausdrücke: `lambda`<a id="orgheadline50"></a></b></div>

-   *Ausdruck*, der eine *anonyme Funktion* vereinbart
    -   Keine Anweisung!
    -   Präziser: der ein Funktionsobjekt erzeugt und eine Referenz auf
        dieses Objekt zurückgibt
-   Eigenschaften
    -   Anonyme Funktion darf beliebig viele Parameter haben
    -   Aber die Funktion darf nur aus einem *Ausdruck* bestehen; sie darf
        *keine Anweisungen* nutzen
-   Syntax:
    -   `lambda Liste von Parameter : Zu berechnender Ausdruck`

<div class="definition-content" style="border:3px; border-style:solid; border-color:#5da9e9; padding: 5px; margin-top: 0; border-radius: 0px 0px 15px 15px; width: 100%; text-align:left"></div>

# &lambda;-Ausdrücke &#x2013; Beispiele

## Funktion, die drei Zahlen addiert<a id="orgheadline52"></a>

In [ ]:
adder = lambda a, b, c : a + b + c 
print(adder(1, 2, 3))

## `lambda` in Factory<a id="orgheadline53"></a>

Viel besser lesbare Closure-Version: 

In [ ]:
def vervielfacher_fabrik(n):
    return lambda x: x*n

doppler = vervielfacher_fabrik(2)
fuenffacher = vervielfacher_fabrik(5) 

print(doppler("Mehrfach ausgeben! "))
print(fuenffacher("Mehrfach ausgeben! "))

# &lambda;-Ausdrücke &#x2013; Funktionen an Objekte übergeben

-   Objekte brauchen manchmal eine Funktion, die sie aufrufen können 
    -   Häufig: GUI-Frameworks
-   Funktion wird nur als Referenz in Objekt gespeichert
-   Funktion bei Methodenaufruf bei `lambda` erzeugen

In [ ]:
class SomeClass:
    def __init__(self, name, fct):
        self.name = name
        self.fct = fct

    def do(self):
        self.fct()


o = SomeClass(name="somename",
              fct = lambda: some expression here)
o.do()

<h1>Overview</h1> 

1. Überblick
1. Verschachtelte Funktionen
1. Closures
1. Anonyme Funktionen: lambda-Ausdrücke
1. **<font color="red">Funktionaler Programmierstil</font>**
1. Generatoren
1. Decorators
1. Zusammenfassung

# Beobachtung: Funktionen sind first-class citizens

-   Python behandelt Funktionen wie ganz normale Objekte, mit allerlei
    Operationen darauf
    -   Funktionen als Parameter, als Rückgabewerte, merken mittels Namen
        oder Datenstrukturen, &#x2026;
-   Solche Funktionen sind *first-class citizens*
-   Dies erlaubt *funktionale Programmierung* als Programmierparadigma

# Funktionale Programmierung &#x2013; Typische Merkmale

-   Funktionen sind first-class citizens
    -   Impliziert: Funktionen können auf Funktionen angewendet werden
-   Rekursion
-   Keine Seiteneffekte

# Beispiel: map, filter

-   Häufiger Stil in funktionalen Sprachen:
    -   Daten in Listen ablegen
    -   Listen filtern gemäß einer Filterfunktion
    -   Funktion auf jedes Element einer Aufzählung (iterable) anwenden
    -   Liste zu einem einzelnen Wert reduzieren
-   Sog. **map/filter/reduce**-Muster

# Iterables

-   Idee nicht nur auf Listen eingeschränkt
-   Warum nicht auf Tuple, Mengen, &#x2026; ?
-   Alles, was man der Reihe nach durchlaufen kann
-   Formalisiert: Klasse `Iterable`
    -   Unterklassen: `list`, `str`, `tuple`, `dict`, Dateien, &#x2026;

# `filter`

-   `filter`: eingebaute Funktion
-   Signatur: `filter(function, iterable)`
    -   Wende die Funktion `function` auf jedes Element einer Aufzählung an
    -   Resultat: Aufzählung mit den Elementen, bei denen die Funktion
        `True` ergab

## Beispiel<a id="orgheadline60"></a>

In [ ]:
l = [1, 2, 3, 4, 5]
l2 = filter(lambda x: x % 2 == 1, l)
print(list(l2))

# `map`

-   `map`: eingebaute Funktion
-   Signatur: `map(function, iterable)`
    -   Wende die Funktion `function` auf jedes Element eines Objektes an,
        dass zu einer von `iterable` abgeleiteten Klasse gehört
    -   Resultat: Aufzählung der Funktionsergebnisse

## Beispiel<a id="orgheadline62"></a>

In [ ]:
l = [1, 2, 3, 4, 5]
l2 = map(lambda x: x*x, l)
print(list(l2))

print(" -- ".join(map(str,l)))

## Anmerkung<a id="orgheadline63"></a>

Konvertierung in Liste nur nötig für `print`

# Weitere Operationen auf iterables

Reichlich Beispiele, siehe Übungen: 

-   zip
-   reduce

# &lambda;-Ausdrücke und flexible Argumente

Eine Funktion, die beliebig viele Argumente beliebigen Types mit & dazwischen ausgibt? 

In [ ]:
out = lambda *x: print(" & ".join(map(str, x)))
out(1, 2, 3, "hallo", "gp1")

# Vergleich: list comprehensions und map/filter

-   Die Beispiele oben sind in Python einfacher und kompakter als list
    comprehensions aufzuschreiben
-   Allerdings: dadurch wird die komplette Liste erzeugt &#x2013; Sinnvoll?

In [ ]:
l = [x for x in range(100000) if x < 2][0:1]
print(l)

# Evolution funktionaler Sprache: Großvater LISP

<center>![img](./figures/lisp_cycles.png "LISP")</center>

# Rein funktionale Sprachen

<center>![img](./figures/haskell.png "Popularität reiner funktionaler Sprachen")</center>

<h1>Overview</h1> 

1. Überblick
1. Verschachtelte Funktionen
1. Closures
1. Anonyme Funktionen: lambda-Ausdrücke
1. Funktionaler Programmierstil
1. **<font color="red">Generatoren</font>**
1. Decorators
1. Zusammenfassung

# Funktionen auf *große* Listen anwenden?

-   Große Eingaben brauchen viel Platz
    -   Ggf. kann das Erzeugen der Eingabe schon allen Speicherplatz
        verbrauchen
-   Mit endlichen Listen kann man keine unendlichen Folgen erzeugen
    -   Beispiel: Alle Primzahlen
-   Vielleicht

## Idee: Werte nach und nach erzeugen<a id="orgheadline70"></a>

-   Statt komplette Liste zu berechnen, nur jeweils den *nächsten* Wert
-   Auf Aufforderung &#x2013; als Funktionsaufruf?

# Werte auf Aufforderung &#x2013; Skizze?

In [ ]:
def magic_prime_production(start):
    if erster Aufruf: 
        letzte_zahl = start
    res = nächste Primzahl größer als letzte_zahl
    letzte_zahl = res
    return res

# Nutzung in for Schleife??
for p in magic_prime_production():
    print(p)

# Werte auf Aufforderung &#x2013; Realisierung?

-   Mit Funktionsaufruf wie bisher geht das so nicht
-   Funktion hat keine Vorstellung, was der *letzte Wert* ist
-   Wir müssten das der Funktion beibringen? &#x2026;?
    -   Analogie: Closures? Hat sich auch Zustand gemerkt?

## Realisierung: Closure-artig<a id="orgheadline73"></a>

-   Mit Closure-artiger Zustandsspeicherung lässt sich das Problem lösen
-   Siehe auch Programmieraufgabe!
-   Aber syntaktisch unhandlich &#x2013; also Anweisung!

# Werte auf Aufforderung &#x2013; Generatoren

Syntaktischer Zucker: Generatoren 

-   Schlüsselwort: `yield` und Generatorenfunktionen
-   Idee: Ersetze in einer Funktion  `return` durch `yield` 
    -   Liefert Wert an Aufrufer
    -   **und** merkt sich Zustand für nächsten Aufruf!
        -   Zustand: Werte lokaler Namen; Zeilennummer; &#x2026;

# Generator: Einfaches Beispiel

Generator wird die Werte 1, 2, 3 liefern. Danach nichts. 

In [ ]:
def gen():
    yield 1
    yield 2
    yield 3

# Generatoren: Nutzung

-   Aufruf: Unterscheidung erster und folgende Aufrufe notwendig
    -   Erster Aufruf: normaler Funktionsaufruf, initialisiert
        -   Gibt ein Generator-Objekt
    -   Folgender Aufruf: Funktion `next` auf dieses Generator-Objekt
        anwenden

# Generator: Einfaches Beispiel &#x2013; Nutzung

Generator wird die Werte 1, 2, 3. Danach `StopIteration`-Exception. 

In [ ]:
def gen():
    yield 1
    yield 2
    yield 3

g = gen()
print(g)
print(next(g))
print(next(g))
print(next(g))
try:
    print(next(g))
except StopIteration:
    print("da kommt nichts mehr")

# Generator: Nutzung in Schleife

Spannender Punkt: Generatoren können wie Iterables in Schleife benutzt
werden 

-   Syntax dadurch deutlich einfacher
-   `StopIteration` führt zum Schleifenende

In [ ]:
def gen():
    yield 1
    yield 2
    yield 3

for v in gen():
    print(v)

# Beispiel: Unendlicher Generator

Produziere **alle** Vielfachen von 3 ab Startwert: 

In [ ]:
def gen(start):
    # wir wollen alle Vielfache, also Endlosschleife: 
    while True:
        if start % 3 == 0:
            yield start
            print("Restarting with value: ", start)
        start += 1

for v, i in zip(gen(16), range(5)):
    print("Wert: ", v)

## Verständnisfragen<a id="orgheadline80"></a>

-   Wozu `range` und `zip` in Beispiel?
-   Wann werden Aufrufe von `gen` durchgeführt?
-   Wieviele Schleifendurchläufe?

# Beispiel: Generator als Closure

In [ ]:
def gengen(multiplier):
    def gen(start):
        while True:
            if start % multiplier == 0:
                yield start
            start += 1

    return gen

dreis = gengen(3)
viers = gengen(4)

for d, v, i in zip(dreis(10), viers(17), range(2)):
    print (d, v)

## Ausgabe?<a id="orgheadline82"></a>

In [ ]:
pingo_title = "WAs wird hier ausgebenen?" 
pingo_type = "single"
pingo_questions = ["12, 15 und 20, 24", "12, 20 und 15, 24", "10, 17 und 13, 21"]
pingo_duration = "60"

%pingo

# Wesentlicher Punkt: Speicherverbrauch

-   Natürlich kann man das auch über eine list comprehension
    hinschreiben
-   Aber Speicherverbrauch:
    -   List comprehension erzeugt die **gesamte Liste** vor Verarbeitung;
        muss im Speicher stehen
    -   Generator: nur Platz für eine einzige Iteration notwendig
        -   Also: `start`, und die Closure für den Generator

# Generators für List comprehensions

Kombination der Vorteile? 

-   Elegante Syntax der List comprehension?
-   Ohne den Speicherverbrauch?

## Generatoren statt Listen!<a id="orgheadline85"></a>

Kleine syntaktische Änderung: Wir ersetzen `[]` durch `()` in
Schleifen

In [ ]:
print("Mit Liste: ")  
for i in [x for x in range(17, 40) if x % 3 == 0]:
    print(i, end=", ")

print("
Mit Generator: ")  
for i in (x for x in range(17, 40) if x % 3 == 0):
    print(i, end=", ")

<h1>Overview</h1> 

1. Überblick
1. Verschachtelte Funktionen
1. Closures
1. Anonyme Funktionen: lambda-Ausdrücke
1. Funktionaler Programmierstil
1. Generatoren
1. **<font color="red">Decorators</font>**
1. Zusammenfassung

# Ausgangspunkt

Wir haben folgende Eigenschaften kennengelernt: 

-   Funktionen sind first-class citizens in Python
-   Funktionsnamen sind eigentlich auch nur Namen, die  ein
    Funktionsobjekt referenzieren
-   Solche Referenzen können aus Funktionen zurückgegeben werden
-   Und Referenzen können  Funktionsnamen zugewiesen werden

Was kann man damit anstellen? 

# Überlegung 1: Wir können aus einer Funktion eine neue Funktion bauen

-   Beispiel: Wir möchten die Ausgaben einer Funktion `f` mit `==` einrahmen
-   Ansatz: Baue eine neue Funktion, die
    -   `=` ausgibt
    -   `f` aufruft
    -   und nochmal `=` ausgibt
-   Diese neue Funktion bauen wir als Closure und geben eine Referenz
    darauf zurück

Schauen wir uns den Code in mehreren Schritten an! 

# Eingerahmte Ausgabe &#x2013; Version 1

Version 1: Eine einfache factory 

In [ ]:
def f():
    print("Ausgabe von f")

def einrahmer_fabrik():
    def einrahmer():
        print("=====")
        f()
        print("=====")

    return einrahmer

f_neu = einrahmer_fabrik()
f_neu()

# Eingerahmte Ausgabe &#x2013; Version 2

Version 1 ist unnötig speziell

-   Factory-Muster kann doch Parameter nehmen (Closures!)

In [ ]:
def f():
    print("Ausgabe von f")

def g():
    print("Ausgabe von g")

def einrahmer_fabrik(fct):
    def einrahmer():
        print("=====")
        fct()
        print("=====")

    return einrahmer

f_neu = einrahmer_fabrik(f) 
g_neu = einrahmer_fabrik(g) 

f_neu() 
g_neu()

# Eingerahmte Ausgabe &#x2013; Version 3

Den einrahmenden Text kann man auch zum Parameter machen

In [ ]:
def f():
    print("Ausgabe von f")

def g():
    print("Ausgabe von g")

def einrahmer_fabrik(fct, s="==========="):
    def einrahmer():
        print(s)
        fct()
        print(s)

    return einrahmer

f_neu = einrahmer_fabrik(f, "***********") 
g_neu = einrahmer_fabrik(g) 

f_neu() 
g_neu()

# Eingerahmte Ausgabe &#x2013; Version 4

Brauchen wir die ursprüngliche Version von `f` noch? Nein? 

-   Dann brauchen wir auch keinen neuen Namen!

In [ ]:
def f():
    print("Ausgabe von f")

def einrahmer_fabrik(fct, s="==========="):
    def einrahmer():
        print(s)
        fct()
        print(s)

    return einrahmer

# Wir lassen den Namen f auf ein neues Funktionsobjekt verweisen:
f = einrahmer_fabrik(f, "***********") 

f()

# Muster

Dieses Muster tritt häufig auf, ist extrem nützlich 

-   Einer Funktion wird neues Verhalten hinzugefügt, ohne dass dies für einen
    Nutzer der Funktion sichtbar ist!
-   Die Signatur der Funktion verändert sich nicht; nicht einmal der Name
-   Nützlich, um vorhandenen Code anzupassen

# Dekoration (*decorator*)

<div class="definition-head" style="border:3px; border-style:solid; border-color:#5da9e9; padding: 5px; background-color:#5da9e9;  line-height:1.5em; text-align:center; border-radius: 15px 15px 0px 0px; margin-bottom: 0; width: 100%;"><b>Definition: Das Decorator-Muster<a id="orgheadline94"></a></b></div>

Das Decorator-Muster beschreibt eine typische Struktur, mit der eine
Funktion mit weiterer Funktionalität versehen werden kann, ohne die
Funktion oder den nutzenden Code zu verändern. 

<div class="definition-content" style="border:3px; border-style:solid; border-color:#5da9e9; padding: 5px; margin-top: 0; border-radius: 0px 0px 15px 15px; width: 100%; text-align:left"></div>

# Decorators: Anwendungen

-   Vorhandenen Code verändern, patchen
-   Fehlersuche, Leistungsmessung
-   Komplexere Code-Strukturen
    -   GUI-Frameworks
        -   Beispiel: einer Funktion zum Zeichnen eines Fenster werden als
            decorators Funktionen zum Zeichnen von Scrollbars hinzugefügt
            -   Funktion selbst und Aufrufer merken nicht, dass das Fenster
                Scrollbars hat
    -   Webframework (Beispiel: Django)
        -   Die Handhabung von URLs wird durch Dekoratoren angepasst

# Decorators für Methoden

-   Methoden einer Klasse sind Funktion
-   Also können sie dekoriert werden

# Decorators: Syntaktischer Zucker

-   Decorators stellen sich als extrem nützlich und vielseitig heraus
-   Aber auch unhandlich 
    -   Beispiel oben: Die Funktion `f` wird 3x erwähnt? DRY?
    -   Die Dekoration kann an ganz anderer Stelle geschehen;
        unübersichtlich
-   Kompakter? Sprachunterstützung?

# Decorators: Syntaktischer Zucker (2)

Syntax: 

-   Die dekorierende Funktion wird wie üblich vereinbart
-   Sie nimmt einen Parameter: die aufzurufende (zu dekorierende)
    Funktion
-   Die zu dekorierende Funktion wird durch voranstellen von
    **@Dekoratorfunktion** dekoriert

# Decorator: kompakte Syntax &#x2013; Beispiel

In [ ]:
def einrahmer_fabrik(fct):
    def einrahmer():
        print("=====")
        fct()
        print("=====")

    return einrahmer

@einrahmer_fabrik
def f():
    print("Ausgabe von f")

f()

# Decorator: kompakte Syntax &#x2013; Beispiel mit typischen Namen

Die `_fabrik`-Terminologie wird bei Dekoratoren typischerweise
weggelassen 

In [ ]:
def einrahmer(fct):
    def _einrahmer():
        print("=====")
        fct()
        print("=====")

    return _einrahmer

@einrahmer
def f():
    print("Ausgabe von f")

f()

# Decorators: Syntaktischer Zucker (3)

`@`-Syntax:

-   Kompakt, leicht lesbar, elegant, häufig benutzt
    -   Sie werden kaum ein nicht-triviales Python-Programm ohne
        decorators finden
-   Nachteil: Parameter an dekorierende Funktion übergeben ist
    umständlich

# @-Decorators: Beispiel Benchmarking

Wie bestimmt man Zeitverbrauch einer Funktion? 

-   Man schaut in einem Dekorator vor und nach dem Aufruf der Funktion
    auf die Uhr!

In [ ]:
import time

def benchmark(fct):
    def _benchmark(*args, **kwargs):
        tstart = time.time()
        r = fct(*args, **kwargs)
        tend = time.time()

        print("{} (args: {}, kwargs: {}) verbrauchte {} Sekunden".format(
            fct.__name__, args, kwargs, tend-tstart))

        return r 

    return _benchmark

@benchmark 
def f1(s):
    print("Anfang f1")
    time.sleep(s)
    print("Ende f1")


f1(0.2)
f1(0.6)

# @-Decorator: Property

Tatsächlich kennen wir schon einen Decorator: Property 

-   Bisher nur als eingebaute Funktion `property`
-   Es fehlt also nur noch der Zucker
    -   Getter: mit `@property` dekorieren; Funktionsname wird Name der
        Property
    -   Setter: mit `@Propertyname.setter` dekorieren; Funktionsname muss
        Name der Property sein, mit value

In [ ]:
class Temperature:
    def __init__(self, temp=0):
        # auch Methoden der Klasse greifen normal auf das Attribut zu: 
        self._temperature = temp

    @property
    def temperature(self):
        print("Lesender Zugriff")
        return self._temperature

    @temperature.setter
    def temperature(self, value):
        print("Schreibender Zugriff")
        if value < 0:
            raise ValueError("Keine negativen Temperaturen in Kelvin!")
        else:
            self._temperature = value

t1 = Temperature(17)
print(t1.temperature) 
t1.temperature = 42 
print(t1.temperature)

# @-Decoratoren verketten?

Siehe auch: [Quelle des Beispiels](http://www.programiz.com/python-programming/decorator)

In [ ]:
def star(func):
    def inner(*args, **kwargs):
        print("*" * 30)
        func(*args, **kwargs)
        print("*" * 30)
    return inner

def percent(func):
    def inner(*args, **kwargs):
        print("%" * 30)
        func(*args, **kwargs)
        print("%" * 30)
    return inner

@star
@percent
def printer(msg):
    print(msg)

printer("Hallo GP1!")

# Decorators: Ausblick

-   Das Dekorieren von Methoden einer Klasse ist typisch (siehe auch 
    `@property`)
-   Man kann auch Klassen zu decorators machen, nicht nur Funktionen
-   Siehe auch [eine lange Liste mit Beispielen](https://wiki.python.org/moin/PythonDecoratorLibrary)

# War doch gar nicht so schwer

<center>![img](./figures/phd-hard-code.png "Master of a tiny universe")</center>

<h1>Overview</h1> 

1. Überblick
1. Verschachtelte Funktionen
1. Closures
1. Anonyme Funktionen: lambda-Ausdrücke
1. Funktionaler Programmierstil
1. Generatoren
1. Decorators
1. **<font color="red">Zusammenfassung</font>**

# Zusammenfassung

-   Funktionen können Funktionen definieren
    -   Erfordert Erweiterung der Nachschlageregel für Namen: LEGB
-   Macht ein Programmierparadigma Funktionen zu *first-class citizens*,
    eröffnen sich viele elegante Möglichkeiten
    -   Fabriken / Closures, um Funktionen zur Laufzeit zu erzeugen
    -   Generatoren, um große (unendliche) Aufzählungen bei Bedarf zu
        erzeugen
    -   Decorators, um Funktionen zur Laufzeit zu modifizieren
-   Eine elegante Syntax ist dabei für praktische Nützlichkeit wichtig
    -   `lambda`-Ausdrücke für anonyme Funktionen, `@`-Notation für
        Decorators

# Und nun?

-   Was waren diese seltsamen `import`-Anweisungen??

  